In [1]:
from mlflow.tracking import MlflowClient

In [2]:
TRACKING_SERVER_HOST = "ec2-3-12-83-113.us-east-2.compute.amazonaws.com"

In [3]:
MLFLOW_TRACKING_URI = f"http://{TRACKING_SERVER_HOST}:5000"

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.list_experiments()

[<Experiment: artifact_location='s3://ml-mlflow-artifact/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://ml-mlflow-artifact/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
client.create_experiment(name="model-registry-experiment")

'2'

In [6]:
from mlflow.entities import ViewType

In [7]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: a964ac50494b49e887981254d0e35f46, rmse: 6.1913
run id: ba57b669cb9249079b2c35298ef36314, rmse: 6.2024
run id: ca09e230edfd4ec1a0cad5aa7a620baf, rmse: 6.2134
run id: 1f7b34ae4c7947899c6a8546bb83cff9, rmse: 6.2186
run id: c8f2900884434627b63b9346d6c45524, rmse: 6.2190


In [9]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "a964ac50494b49e887981254d0e35f46"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
2022/06/28 18:10:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1656439855326, current_stage='None', description='', last_updated_timestamp=1656439855326, name='nyc-taxi-regressor', run_id='a964ac50494b49e887981254d0e35f46', run_link='', source='s3://ml-mlflow-artifact/1/a964ac50494b49e887981254d0e35f46/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
run_id = "ba57b669cb9249079b2c35298ef36314"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/06/28 18:12:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1656439967149, current_stage='None', description='', last_updated_timestamp=1656439967149, name='nyc-taxi-regressor', run_id='ba57b669cb9249079b2c35298ef36314', run_link='', source='s3://ml-mlflow-artifact/1/ba57b669cb9249079b2c35298ef36314/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [13]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [14]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1656439855326, current_stage='Production', description='', last_updated_timestamp=1656440149483, name='nyc-taxi-regressor', run_id='a964ac50494b49e887981254d0e35f46', run_link='', source='s3://ml-mlflow-artifact/1/a964ac50494b49e887981254d0e35f46/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [17]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1656439855326, current_stage='Production', description='The model version 1 was transitioned to Production on 2022-06-28', last_updated_timestamp=1656440241769, name='nyc-taxi-regressor', run_id='a964ac50494b49e887981254d0e35f46', run_link='', source='s3://ml-mlflow-artifact/1/a964ac50494b49e887981254d0e35f46/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>